In [ ]:
# Install necessary dependencies.
using Pkg
Pkg.activate(; temp=true)
Pkg.add(["Turing", "DifferentialEquations", "Distributions", "LinearAlgebra", "StatsPlots", "Mooncake", "SciMLSensitivity", "Random"])

In [ ]:
#| echo: false
#| output: false
using Pkg;
Pkg.instantiate();

A basic scientific problem is to mathematically model a system of interest, then compare this model to the observable reality around us.
Such models often involve dynamical systems of differential equations.
In practice, these equations often have unknown parameters we would like to estimate.
The "[forward problem](https://en.wikipedia.org/wiki/Well-posed_problem)" of simulation consists of solving the differential equations for a given set of parameters, while the "[inverse problem](https://en.wikipedia.org/wiki/Inverse_problem)" of parameter estimation uses observed data to infer the unknown model parameters.
Bayesian inference provides a robust approach to parameter estimation with quantified uncertainty.

In [ ]:
using Turing
using DifferentialEquations
# Load StatsPlots for visualizations and diagnostics.
using StatsPlots
using LinearAlgebra
using Distributions
# Set a seed for reproducibility.
using Random
Random.seed!(14);

## The Lotka–Volterra Model

[The Lotka–Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations), also known as the predator–prey equations, are a pair of first-order nonlinear differential equations.
These differential equations are frequently used to describe the dynamics of biological systems in which two species interact, one as a predator and the other as prey.
The populations change through time according to the pair of equations

$$
\begin{aligned}
\frac{\mathrm{d}x}{\mathrm{d}t} &= (\alpha - \beta y(t))x(t), \\
\frac{\mathrm{d}y}{\mathrm{d}t} &= (\delta x(t) - \gamma)y(t),
\end{aligned}
$$

where $x(t)$ and $y(t)$ denote the populations of prey and predator at time $t$, respectively, and $\alpha, \beta, \gamma, \delta$ are positive parameters.
In the absence of predators, the prey population $x$ would increase exponentially at rate $\alpha$ (with dimensions of time^-1^).
However, the predators kill some prey at a rate $\beta$ (prey predator^-1^ time^-1^), which enables the predator population to increase at rate $\delta$ (predators prey^-1^ time^-1^).
Finally, predators are removed by natural mortality at rate $\gamma$ (time^-1^).

We implement the Lotka–Volterra model and simulate it with parameters $\alpha = 1.5$, $\beta = 1$, $\gamma = 3$, and $\delta = 1$ and initial conditions $x(0) = y(0) = 1$.

In [ ]:
# Define Lotka–Volterra model.
function lotka_volterra(du, u, p, t)
    # Model parameters.
    α, β, γ, δ = p
    # Current state.
    x, y = u

    # Evaluate differential equations.
    du[1] = (α - β * y) * x # prey
    du[2] = (δ * x - γ) * y # predator

    return nothing
end

# Define initial-value problem.
u0 = [1.0, 1.0]
p = [1.5, 1.0, 3.0, 1.0]
tspan = (0.0, 10.0)
prob = ODEProblem(lotka_volterra, u0, tspan, p)

# Plot simulation.
plot(solve(prob, Tsit5()))

We generate noisy observations to use for the parameter estimation tasks in this tutorial.
With the [`saveat` argument](https://docs.sciml.ai/DiffEqDocs/stable/basics/common_solver_opts/) to the differential equation solver, we specify that the solution is stored only at `0.1` time units.

To make the example more realistic, we generate data as random [Poisson](https://en.wikipedia.org/wiki/Poisson_distribution) counts based on the "true" population densities of predator and prey from the simulation.
Poisson-distributed data are common in ecology (for instance, counts of animals detected by a camera trap).
We assume the Poisson rate parameter $\lambda$ is proportional to the underlying animal densities, with proportionality constant $q = 1.7$ (representing observation efficiency).

In [ ]:
sol = solve(prob, Tsit5(); saveat=0.1)
q = 1.7
odedata = rand.(Poisson.(q * Array(sol)))

# Plot simulation and noisy observations.
plot(sol, label=["Prey" "Predator"])
scatter!(sol.t, odedata'; color=[1 2], label="")

An even more realistic example could be fitted to the famous hare-and-lynx system using the long-term trapping records of the Hudson's Bay Company.
A Stan implementation of this problem with slightly different priors can be found [here](https://mc-stan.org/learn-stan/case-studies/lotka-volterra-predator-prey.html).
For this tutorial, though, we will stick with simulated data.

## Direct Handling of Bayesian Estimation with Turing

[DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/stable/) is the main Julia package for numerically solving differential equations.
Its functionality is completely interoperable with Turing.jl, which means that we can directly simulate differential equations inside a Turing `@model`.

For the purposes of this tutorial, we choose priors for the parameters that are quite close to the ground truth.
As justification, we can imagine we have preexisting estimates for the biological rates.
Practically, this helps us to illustrate the results without needing to run overly long MCMC chains.

Note we also have to take special care with the ODE solver.
For certain parameter combinations, the numerical solver may predict animal densities that are just barely below zero.
This causes errors with the Poisson distribution, which needs a non-negative mean $\lambda$.
To avoid this happening, we tell the solver to aim for small absolute and relative errors (`abstol=1e-6, reltol=1e-6`).
We also add a fudge factor `ϵ = 1e-5` to the predicted data.
Since `ϵ` is greater than the solver's tolerance, it should overcome any remaining numerical error, making sure all predicted values are positive.
At the same time, it is so small compared to the data that it should have a negligible effect on inference.
If this approach doesn't work, there are some more ideas to try [here](https://docs.sciml.ai/DiffEqDocs/stable/basics/faq/#My-ODE-goes-negative-but-should-stay-positive,-what-tools-can-help?).
In the case of continuous observations (e.g. data derived from modelling chemical reactions), it is sufficient to use a normal distribution with the mean as the data point and an appropriately chosen variance (which can itself also be a parameter with a prior distribution).

In [ ]:
@model function fitlv(data, prob)
    # Prior distributions.
    α ~ truncated(Normal(1.5, 0.2); lower=0.5, upper=2.5)
    β ~ truncated(Normal(1.1, 0.2); lower=0, upper=2)
    γ ~ truncated(Normal(3.0, 0.2); lower=1, upper=4)
    δ ~ truncated(Normal(1.0, 0.2); lower=0, upper=2)
    q ~ truncated(Normal(1.7, 0.2); lower=0, upper=3)

    # Simulate Lotka–Volterra model. 
    p = [α, β, γ, δ]
    predicted = solve(prob, Tsit5(); p=p, saveat=0.1, abstol=1e-6, reltol=1e-6)
    ϵ = 1e-5
    
    # Observations.
    for i in eachindex(predicted)
        data[:, i] ~ arraydist(Poisson.(q .* predicted[i] .+ ϵ))
    end

    return nothing
end

model = fitlv(odedata, prob)

# Sample 3 independent chains with forward-mode automatic differentiation (the default).
chain = sample(model, NUTS(), MCMCSerial(), 1000, 3; progress=false)

The estimated parameters are close to the parameter values the observations were generated with. We can also check visually that the chains have converged.

In [ ]:
plot(chain)

### Data retrodiction

In Bayesian analysis it is often useful to retrodict the data, i.e. generate simulated data using samples from the posterior distribution, and compare to the original data (see for instance section 3.3.2 - model checking of McElreath's book "Statistical Rethinking").
Here, we solve the ODE for 300 randomly picked posterior samples in the `chain`.
We plot the ensemble of solutions to check if the solution resembles the data.
The 300 retrodicted time courses from the posterior are plotted in gray, the noisy observations are shown as blue and red dots, and the green and purple lines are the ODE solution that was used to generate the data.

In [ ]:
plot(; legend=false)
posterior_samples = sample(chain[[:α, :β, :γ, :δ]], 300; replace=false)
for p in eachrow(Array(posterior_samples))
    sol_p = solve(prob, Tsit5(); p=p, saveat=0.1)
    plot!(sol_p; alpha=0.1, color="#BBBBBB")
end

# Plot simulation and noisy observations.
plot!(sol; color=[1 2], linewidth=1)
scatter!(sol.t, odedata'; color=[1 2])

We can see that, even though we added quite a bit of noise to the data the posterior distribution reproduces quite accurately the "true" ODE solution.

## Lotka–Volterra model without data of prey

One can also perform parameter inference for a Lotka–Volterra model with incomplete data.
For instance, let us suppose we have only observations of the predators but not of the prey.
We can fit the model only to the $y$ variable of the system without providing any data for $x$:

In [ ]:
@model function fitlv2(data::AbstractVector, prob)
    # Prior distributions.
    α ~ truncated(Normal(1.5, 0.2); lower=0.5, upper=2.5)
    β ~ truncated(Normal(1.1, 0.2); lower=0, upper=2)
    γ ~ truncated(Normal(3.0, 0.2); lower=1, upper=4)
    δ ~ truncated(Normal(1.0, 0.2); lower=0, upper=2)
    q ~ truncated(Normal(1.7, 0.2); lower=0, upper=3)

    # Simulate Lotka–Volterra model but save only the second state of the system (predators).
    p = [α, β, γ, δ]
    predicted = solve(prob, Tsit5(); p=p, saveat=0.1, save_idxs=2, abstol=1e-6, reltol=1e-6)
    ϵ = 1e-5

    # Observations of the predators.
    data ~ arraydist(Poisson.(q .* predicted.u .+ ϵ))

    return nothing
end

model2 = fitlv2(odedata[2, :], prob)

# Sample 3 independent chains.
chain2 = sample(model2, NUTS(0.45), MCMCSerial(), 5000, 3; progress=false)

Again we inspect the trajectories of 300 randomly selected posterior samples.

In [ ]:
plot(; legend=false)
posterior_samples = sample(chain2[[:α, :β, :γ, :δ]], 300; replace=false)
for p in eachrow(Array(posterior_samples))
    sol_p = solve(prob, Tsit5(); p=p, saveat=0.1)
    plot!(sol_p; alpha=0.1, color="#BBBBBB")
end

# Plot simulation and noisy observations.
plot!(sol; color=[1 2], linewidth=1)
scatter!(sol.t, odedata'; color=[1 2])

Note that here the observations of the prey (blue dots) were not used in the parameter estimation!
Yet, the model can predict the values of $x$ relatively accurately, albeit with a wider distribution of solutions, reflecting the greater uncertainty in the prediction of the $x$ values.

## Inference of Delay Differential Equations

Here we show an example of inference with another type of differential equation: a delay differential equation (DDE).
DDEs are differential equations where derivatives are functions of values at an earlier point in time.
This is useful to model a delayed effect, such as the incubation time of a virus.

Here is a delayed version of the Lotka–Volterra system:

$$
\begin{aligned}
\frac{\mathrm{d}x}{\mathrm{d}t} &= \alpha x(t-\tau) - \beta y(t) x(t),\\
\frac{\mathrm{d}y}{\mathrm{d}t} &= - \gamma y(t) + \delta x(t) y(t),
\end{aligned}
$$

where $\tau$ is a (positive) delay and $x(t-\tau)$ is the variable $x$ at an earlier time point $t - \tau$.

The initial-value problem of the delayed system can be implemented as a `DDEProblem`.
As described in the [DDE example](https://diffeq.sciml.ai/stable/tutorials/dde_example/), here the function `h` is the history function that can be used to obtain a state at an earlier time point.
Again we use parameters $\alpha = 1.5$, $\beta = 1$, $\gamma = 3$, and $\delta = 1$ and initial conditions $x(0) = y(0) = 1$.
Moreover, we assume $x(t) = 1$ for $t < 0$.

In [ ]:
function delay_lotka_volterra(du, u, h, p, t)
    # Model parameters.
    α, β, γ, δ = p

    # Current state.
    x, y = u
    # Evaluate differential equations
    du[1] = α * h(p, t - 1; idxs=1) - β * x * y
    du[2] = -γ * y + δ * x * y

    return nothing
end

# Define initial-value problem.
p = (1.5, 1.0, 3.0, 1.0)
u0 = [1.0; 1.0]
tspan = (0.0, 10.0)
h(p, t; idxs::Int) = 1.0
prob_dde = DDEProblem(delay_lotka_volterra, u0, h, tspan, p);

We generate observations by sampling from the corresponding Poisson distributions derived from the simulation results:

In [ ]:
sol_dde = solve(prob_dde; saveat=0.1)
ddedata = rand.(Poisson.(q .* Array(sol_dde)))

# Plot simulation and noisy observations.
plot(sol_dde)
scatter!(sol_dde.t, ddedata'; color=[1 2], label="")

Now we define the Turing model for the Lotka–Volterra model with a delay, and sample 3 independent chains.

In [ ]:
@model function fitlv_dde(data, prob)
    # Prior distributions.
    α ~ truncated(Normal(1.5, 0.2); lower=0.5, upper=2.5)
    β ~ truncated(Normal(1.1, 0.2); lower=0, upper=2)
    γ ~ truncated(Normal(3.0, 0.2); lower=1, upper=4)
    δ ~ truncated(Normal(1.0, 0.2); lower=0, upper=2)
    q ~ truncated(Normal(1.7, 0.2); lower=0, upper=3)

    # Simulate Lotka–Volterra model.
    p = [α, β, γ, δ]
    predicted = solve(prob, MethodOfSteps(Tsit5()); p=p, saveat=0.1, abstol=1e-6, reltol=1e-6)
    ϵ = 1e-5

    # Observations.
    for i in eachindex(predicted)
        data[:, i] ~ arraydist(Poisson.(q .* predicted[i] .+ ϵ))
    end
end

model_dde = fitlv_dde(ddedata, prob_dde)

chain_dde = sample(model_dde, NUTS(), MCMCSerial(), 300, 3; progress=false)

In [ ]:
plot(chain_dde)

Finally, we plot trajectories of 300 randomly selected samples from the posterior.
Again, the dots indicate our observations, the coloured lines are the "true" simulations without noise, and the gray lines are trajectories from the posterior samples.

In [ ]:
plot(; legend=false)
posterior_samples = sample(chain_dde[[:α, :β, :γ, :δ]], 300; replace=false)
for p in eachrow(Array(posterior_samples))
    sol_p = solve(prob_dde, MethodOfSteps(Tsit5()); p=p, saveat=0.1)
    plot!(sol_p; alpha=0.1, color="#BBBBBB")
end

# Plot simulation and noisy observations.
plot!(sol_dde; color=[1 2], linewidth=1)
scatter!(sol_dde.t, ddedata'; color=[1 2])

The fit is pretty good even though the data was quite noisy to start.

## Scaling to Large Models: Adjoint Sensitivities

Turing's gradient-based MCMC algorithms, such as NUTS, use ForwardDiff by default.
This works well for small models, but for larger models with many parameters, reverse-mode automatic differentiation is often more efficient (see [the automatic differentiation page]({{< meta usage-automatic-differentiation >}}) for more information).

To use reverse-mode AD with differential equations, you need to first load the [SciMLSensitivity.jl package](https://docs.sciml.ai/SciMLSensitivity/stable/), which forms part of SciML's differential equation suite.
Here, 'sensitivity' refers to the derivative of the solution of a differential equation with respect to its parameters.
More details on the mathematical theory that underpins these methods can be found in [the SciMLSensitivity documentation](https://docs.sciml.ai/SciMLSensitivity/stable/sensitivity_math/).

Once SciMLSensitivity has been loaded, you can use one of the AD backends which are compatible with SciMLSensitivity.jl.
For example, if we wanted to use [Mooncake.jl](https://chalk-lab.github.io/Mooncake.jl/stable/), we could run:

In [ ]:
import Mooncake
import SciMLSensitivity

# Define the AD backend to use
adtype = AutoMooncake()

# Sample a single chain with 1000 samples using Mooncake
sample(model, NUTS(; adtype=adtype), 1000; progress=false)

(If SciMLSensitivity is not loaded, the call to `sample` will error.)

SciMLSensitivity has a number of sensitivity analysis algorithms: in this case it will automatically choose a default for you.
You can also manually specify an algorithm by providing the `sensealg` keyword argument to the `solve` function; the existing algorithms are covered in [this page of the SciMLSensitivity docs](https://docs.sciml.ai/SciMLSensitivity/stable/manual/differential_equation_sensitivities/).

For more examples of adjoint usage on large parameter models, consult the [DiffEqFlux documentation](https://docs.sciml.ai/DiffEqFlux/stable/).